In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
from tensorflow.keras.layers import Dropout

In [ ]:
!rm -rf ./logs/ 

In [ ]:
"""
## Prepare the data
"""
batch_size = 128

TRAINING_DIR = '/Users/amitaflalo/Desktop/sudoku/dataset/train'
VAL_DIR = '/Users/amitaflalo/Desktop/sudoku/dataset/val'
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    color_mode='grayscale',
                                                    target_size=(28, 28),
                                                    shuffle=True,
                                                    )
                                                    
validation_generator = train_datagen.flow_from_directory(VAL_DIR, 
                                                        target_size=(28, 28),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        color_mode='grayscale',
                                                        )
                                                                    

num_classes = 9
input_shape = (28, 28, 1)

In [ ]:
STEPS_PER_EPOCH = int( np.ceil( 60885 / batch_size))

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

In [ ]:
"""
## Build the model
"""

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')])

model.summary() #model summary


In [ ]:
"""
## Train the model
"""
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

epochs = 50

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(train_generator, epochs=epochs, steps_per_epoch=STEPS_PER_EPOCH, validation_data=validation_generator, validation_steps=1, callbacks=[tensorboard_callback], shuffle = True)

In [ ]:
# Open an embedded TensorBoard viewer
%tensorboard --logdir logs/fit

In [ ]:
# serialize model to JSON and save the model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save("model(1.7).h5")